In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import boto3
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re

if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_All.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

# load data
train_df = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train = train_df[label_names].values

train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
train_df

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


,document,link,label,doc_len
0,OECD 회원국 중 한국 터키만 상원이 없다,https://factcheck.snu.ac.kr/v2/facts/3285,전혀 사실 아님,7
1,"트럼프 ""100조 무기가 탈레반에"", 사실일까?",https://factcheck.snu.ac.kr/v2/facts/3283,대체로 사실 아님,5
2,대선후보 경선등록 후 완주하지 않으면 독자출마 가능하다,https://factcheck.snu.ac.kr/v2/facts/3281,대체로 사실 아님,7
3,사형 집행해야 사회 안전해진다,https://factcheck.snu.ac.kr/v2/facts/3279,대체로 사실 아님,4
4,사랑제일교회 야외 예배는 방역법을 준수했다,https://factcheck.snu.ac.kr/v2/facts/3278,전혀 사실 아님,5
...,...,...,...,...
4234,여름엔 긴팔 셔츠보다 반팔이 더 시원할까?,https://factcheck.snu.ac.kr/v2/facts/210,절반의 사실,6
4235,단종된 갤럭시노트7의 재고 부품을 활용한 삼성전자의 '갤럭시노트FE'는 신제품일까?,https://factcheck.snu.ac.kr/v2/facts/204,절반의 사실,8
4236,공공와이파이 20만개 설치시 연 4천800억~8천500억원의 데이터요금을 경감할 수...,https://factcheck.snu.ac.kr/v2/facts/191,대체로 사실,9
4237,참여연대 등이 '통신 기본료 1만1천원 폐지' 공약 이행을 주장하고 있다. L...,https://factcheck.snu.ac.kr/v2/facts/187,논쟁중,18


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [6]:
# train_data = pd.read_table('ratings_train.txt')

In [2]:
print(len(train_df))

4239


In [3]:
train_data = train_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [4]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [5]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [6]:
from gensim.models import FastText
model = FastText(sentences = tokenized_data, window = 5, min_count = 5, workers = 4, sg = 1)

model.save('similar_keyword_model_SNU')

In [7]:
loaded_model = FastText.load('similar_keyword_model_SNU')
print(loaded_model.wv.most_similar('김정은', topn=5))

[('유니클로', 0.9996781945228577), ('공공기관', 0.999670147895813), ('플라스틱', 0.9996676445007324), ('줄어들다', 0.9996495842933655), ('불법체류자', 0.9996414184570312)]


In [8]:
from gensim.models import FastText
loaded_model = FastText.load('similar_keyword_model_SNU')

a = loaded_model.wv.most_similar('코로나', topn=5)
a

[('감염', 0.9859572052955627),
 ('바이러스', 0.9824109673500061),
 ('신종', 0.980414628982544),
 ('코로나바이러스', 0.977348804473877),
 ('맞다', 0.9728062152862549)]

In [9]:
a[0][0]

'김정일'

In [13]:
blist = []
for i in range(len(a)):
    blist.append(a[i][0])
    
blist

['김정일', '정은', '이수현', '김정태', '김정훈']